---
# INFO-H600 - Computing Foundations of Data Sciences

## Team 14 : 

Roman Lešický, Theo Abraham, Kevin Straatman, Lara Hansen, Grégoire Van den Eynde and Nicolas Roux

Version of python : 3.11.14 | packaged by conda-forge 

---

# Library:

###### The download environment.txt is present within the Github repository of the project https://github.com/RomanLesicky/Data_Science_Project_INFO_H600

In [ ]:
import pspark

---

# Overview of the project :

# `WIP`

---

# Step 1:

`TBD`

# See word for the reasons why I chose Spark over Dusk 

----

# Step 2:

---